This notebook compares Qemistree's fingerprint-based pipeline to cosine-based pipelines using three public datasets: Evaluation dataset [(MSV000083306)](ftp://massive.ucsd.edu/MSV000083306), Cheetah fecal metabolome dataset [(MSV000082969)](ftp://massive.ucsd.edu/MSV000082969/), and Global FoodOmics [(MSV000085226)](ftp://massive.ucsd.edu/MSV000085226/).

In [7]:
import qiime2
from qiime2 import Artifact, Metadata
import pandas as pd
import q2_qemistree
from q2_qemistree import get_classyfire_taxonomy
import numpy as np
import biom

from skbio import DistanceMatrix
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage
from skbio import TreeNode
from skbio.stats.distance import permanova
import itertools

import qiime2.plugins.diversity.actions as q2_diversity
import qiime2.plugins.feature_table.actions as q2_feature_table

In [8]:
# compare CSI & MS/MS agreement at each ClassyFire taxonomic level
def qc_csi_classyfire(classified_fdata_csi, classified_fdata_ms2):
    levels = ['kingdom', 'superclass', 'class', 'subclass', 'direct_parent']
    for level in levels:
        num_match = []
        num_errors = []
        num_mismatch = []
        for idx in classified_fdata_csi.index:
            csi = classified_fdata_csi.loc[idx, level]
            ms2 = classified_fdata_ms2.loc[idx, level]
            errors = ['unclassified', 'unexpected server response', 'SMILE parse error', np.nan]
            if csi not in errors and ms2 not in errors:
                if classified_fdata_csi.loc[idx, level] == classified_fdata_ms2.loc[idx, level]:
                    num_match.append(idx)
                else:
                    num_mismatch.append(idx)
            else:
                num_errors.append(idx)
                
        print(level+':', len(num_match)*100.0/(len(classified_fdata_csi)-len(num_errors)))
        print("No. of annotations compared", len(classified_fdata_csi)-len(num_errors))

In [9]:
def load_mf(fn, index='#SampleID', sep='\t'):
    _df = pd.read_csv(fn, sep=sep, dtype='str', na_values=[], keep_default_na=False)
    _df.set_index(index, inplace=True)
    return _df

## Evaluation Dataset

- The mass-spectrometry files were processed using Mzmine2 (version 2.38) followed by feature-based molecular networking (FBMN) in GNPS: [GNPS task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=044e981ff0d84246ae5c91ef3db643a8)

- The resulting files were analysed using q2-qemistree in GNPS: [GNPS Task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=8ca56d6e33bc4106b46ba5e3510c91cb)

In [4]:
# total number of features detected in MZmine2
fbmn_ftable = Artifact.load('data/comparison-to-cosine/EvaluationData/'
                            'FBMN/qiime2_output/qiime2_table.qza').view(biom.Table)
len(fbmn_ftable.ids(axis='observation'))

7032

In [5]:
# GFOP: feature data with annotations from CSI:FingerID only
! qiime qemistree make-hierarchy \
  --i-csi-results data/comparison-to-cosine/EvaluationData/Qemistree/output_folder/fingerprints.qza \
  --i-feature-tables data/comparison-to-cosine/EvaluationData/FBMN/qiime2_output/qiime2_table.qza \
  --o-tree data/comparison-to-cosine/EvaluationData/tree_fingerprints.qza \
  --o-feature-table data/comparison-to-cosine/EvaluationData/feature_table_hashed.qza \
  --o-feature-data data/comparison-to-cosine/EvaluationData/feature_data_no_specmatch.qza

# structural annotations by CSI
fdata = Artifact.load('data/comparison-to-cosine/EvaluationData/'
                      'feature_data_no_specmatch.qza').view(pd.DataFrame)
print(fdata.shape)
fdata_csi = fdata[fdata['csi_smiles'] != 'missing']
print(fdata_csi.shape)

Saved Phylogeny[Rooted] to: data/comparison-to-cosine/EvaluationData/tree_fingerprints.qza
Saved FeatureTable[Frequency] to: data/comparison-to-cosine/EvaluationData/feature_table_hashed.qza
Saved FeatureData[Molecules] to: data/comparison-to-cosine/EvaluationData/feature_data_no_specmatch.qza
(4399, 7)
(4025, 7)


In [6]:
# feature data with annotations from CSI:FingerID + ms/ms spectral matches
classified_fdata =  Artifact.load('data/comparison-to-cosine/EvaluationData/Qemistree/'
                                  'output_folder/classified-feature-data-updated.qza').view(pd.DataFrame)
classified_fdata_ms2 = classified_fdata[classified_fdata['ms2_smiles'] != 'missing']

# subset of features with both MS/MS matches and CSI:FingerID annotations
features_csi_ms2 = classified_fdata_ms2.index.intersection(fdata_csi.index)
len(features_csi_ms2)

331

In [11]:
# total number of strutural annotations (MS/MS & CSI:FingerID)
classified_fdata[classified_fdata['smiles'] != 'missing'].shape

(4033, 19)

In [12]:
# total number of classyfire annotations for the above structures
classified_fdata['kingdom'].value_counts()

Organic compounds    2378
unclassified         2020
SMILE parse error       1
Name: kingdom, dtype: int64

In [13]:
# run classyfire for csi (ms2 structures have classyfire assignment by default)
classified_fdata_csi = get_classyfire_taxonomy(fdata_csi.loc[features_csi_ms2])

In [14]:
qc_csi_classyfire(classified_fdata_csi, classified_fdata_ms2)

kingdom: 100.0
No. of annotations compared 302
superclass: 85.76158940397352
No. of annotations compared 302
class: 70.19867549668874
No. of annotations compared 302
subclass: 41.281138790035584
No. of annotations compared 281
direct_parent: 25.827814569536425
No. of annotations compared 302


## Cheetah Fecal Metabolome

- The mass-spectromtery files were processed using Mzmine2 (version 2.38) followed by feature-based molecular networking (FBMN) in GNPS: [GNPS task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=8cc48b102b3d41179e0456fd73e89e0d)

- The resulting files were analysed using q2-qemistree in GNPS: [GNPS Task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=9c8fa1553d1a43e08e1dca30723c868c)

In [16]:
# total number of features detected in MZmine2
fbmn_ftable = Artifact.load('data/comparison-to-cosine/FecalCheetah/'
                            'FBMN/qiime2_output/qiime2_table.qza').view(biom.Table)
len(fbmn_ftable.ids(axis='observation'))

1165

In [17]:
# GFOP: feature data with annotations from CSI:FingerID only
! qiime qemistree make-hierarchy \
  --i-csi-results data/comparison-to-cosine/FecalCheetah/Qemistree/output_folder/fingerprints.qza \
  --i-feature-tables data/comparison-to-cosine/FecalCheetah/FBMN/qiime2_output/qiime2_table.qza \
  --o-tree data/comparison-to-cosine/FecalCheetah/tree_fingerprints.qza \
  --o-feature-table data/comparison-to-cosine/FecalCheetah/feature_table_hashed.qza \
  --o-feature-data data/comparison-to-cosine/FecalCheetah/feature_data_no_specmatch.qza

# structural annotations by CSI
fdata = Artifact.load('data/comparison-to-cosine/FecalCheetah/'
                      'feature_data_no_specmatch.qza').view(pd.DataFrame)
print(fdata.shape)
fdata_csi = fdata[fdata['csi_smiles'] != 'missing']
print(fdata_csi.shape)

Saved Phylogeny[Rooted] to: data/comparison-to-cosine/FecalCheetah/tree_fingerprints.qza
Saved FeatureTable[Frequency] to: data/comparison-to-cosine/FecalCheetah/feature_table_hashed.qza
Saved FeatureData[Molecules] to: data/comparison-to-cosine/FecalCheetah/feature_data_no_specmatch.qza
(684, 7)
(513, 7)


In [20]:
# feature data with annotations from CSI:FingerID + ms/ms spectral matches
classified_fdata =  Artifact.load('data/comparison-to-cosine/FecalCheetah/Qemistree/'
                                  'output_folder/classified-feature-data-updated.qza').view(pd.DataFrame)
classified_fdata_ms2 = classified_fdata[classified_fdata['ms2_smiles'] != 'missing']

# subset of features with both MS/MS matches and CSI:FingerID annotations
features_csi_ms2 = classified_fdata_ms2.index.intersection(fdata_csi.index)
len(features_csi_ms2)

121

In [21]:
# total number of strutural annotations (MS/MS & CSI:FingerID)
classified_fdata[classified_fdata['smiles'] != 'missing'].shape

(550, 14)

In [22]:
# total number of classyfire annotations for the above structures
classified_fdata['kingdom'].value_counts()

Organic compounds    406
unclassified         278
Name: kingdom, dtype: int64

In [23]:
# run classyfire for csi (ms2 structures have classyfore by default)
classified_fdata_csi = get_classyfire_taxonomy(fdata_csi.loc[features_csi_ms2])

In [24]:
qc_csi_classyfire(classified_fdata_csi, classified_fdata_ms2)

kingdom: 100.0
No. of annotations compared 112
superclass: 70.53571428571429
No. of annotations compared 112
class: 65.17857142857143
No. of annotations compared 112
subclass: 58.490566037735846
No. of annotations compared 106
direct_parent: 35.714285714285715
No. of annotations compared 112


## Global FoodOmics

- The mass-spectromtery files were processed using Mzmine2 (version 2.38) followed by feature-based molecular networking (FBMN) in GNPS: [GNPS task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=7718fe06d30c4756af7b8638120a292b)

- The resulting files were analysed using q2-qemistree in GNPS: [GNPS Task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=831d6f00d1544d97b32850e0bdf6c401)

In [25]:
# total number of features detected in MZmine2
fbmn_ftable = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/'
                            'FBMN/qiime2_output/qiime2_table.qza').view(biom.Table)
len(fbmn_ftable.ids(axis='observation'))

1634

In [26]:
# GFOP: feature data with annotations from CSI:FingerID only
! qiime qemistree make-hierarchy \
  --i-csi-results data/comparison-to-cosine/GlobalFoodOmics/Qemistree/output_folder/fingerprints.qza \
  --i-feature-tables data/comparison-to-cosine/GlobalFoodOmics/FBMN/qiime2_output/qiime2_table.qza \
  --o-tree data/comparison-to-cosine/GlobalFoodOmics/tree_fingerprints.qza \
  --o-feature-table data/comparison-to-cosine/GlobalFoodOmics/feature_table_hashed.qza \
  --o-feature-data data/comparison-to-cosine/GlobalFoodOmics/feature_data_no_specmatch.qza

# structural annotations by CSI
fdata_gfop = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/'
                           'feature_data_no_specmatch.qza').view(pd.DataFrame)
print(fdata_gfop.shape)
fdata_gfop_csi = fdata_gfop[fdata_gfop['csi_smiles'] != 'missing']
print(fdata_gfop_csi.shape)

Saved Phylogeny[Rooted] to: data/comparison-to-cosine/GlobalFoodOmics/tree_fingerprints.qza
Saved FeatureTable[Frequency] to: data/comparison-to-cosine/GlobalFoodOmics/feature_table_hashed.qza
Saved FeatureData[Molecules] to: data/comparison-to-cosine/GlobalFoodOmics/feature_data_no_specmatch.qza
(663, 7)
(599, 7)


In [32]:
# feature data with annotations from CSI:FingerID + ms/ms spectral matches
classified_fdata_gfop =  Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/Qemistree/'
                                       'output_folder/classified-feature-data-updated.qza').view(pd.DataFrame)
classified_fdata_gfop_ms2 = classified_fdata_gfop[classified_fdata_gfop['ms2_smiles'] != 'missing']

# subset of features with both MS/MS matches and CSI:FingerID annotations
features_csi_ms2 = classified_fdata_gfop_ms2.index.intersection(fdata_gfop_csi.index)
len(features_csi_ms2)

57

In [33]:
# total number of strutural annotations (MS/MS & CSI:FingerID)
classified_fdata_gfop[classified_fdata_gfop['smiles'] != 'missing'].shape

(601, 14)

In [34]:
# total number of classyfire annotations for the above structures
classified_fdata_gfop['kingdom'].value_counts()

Organic compounds    553
unclassified         109
SMILE parse error      1
Name: kingdom, dtype: int64

In [30]:
# run classyfire for csi (ms2 structures have classyfore by default)
classified_fdata_gfop_csi = get_classyfire_taxonomy(fdata_gfop_csi.loc[features_csi_ms2])

In [35]:
qc_csi_classyfire(classified_fdata_gfop_csi, classified_fdata_gfop_ms2)

kingdom: 100.0
No. of annotations compared 55
superclass: 78.18181818181819
No. of annotations compared 55
class: 65.45454545454545
No. of annotations compared 55
subclass: 66.66666666666667
No. of annotations compared 48
direct_parent: 58.18181818181818
No. of annotations compared 55


### I. Comparison to UniFrac using Cosine-based tree 

The following feature-based molecular networking (FBMN) job was run on GNPS to obtain all-vs-all cosine scores: [GNPS task](https://gnps.ucsd.edu/ProteoSAFe/status.jsp?task=69d297bf9f7e41849d48d6728eb42f5f)

**Note:** Cosine similarity score < 0.3 is set to 0

In [36]:
classified_fdata = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/Qemistree/output_folder/'
                                 'classified-feature-data-updated.qza').view(pd.DataFrame)
classified_fdata.head()

,#featureID,csi_smiles,ms2_smiles,ms2_library_match,parent_mass,retention_time,table_number,smiles,structure_source,kingdom,superclass,class,subclass,direct_parent
id,,,,,,,,,,,,,,
e459465fab00fe37f66bbd801e1abe23,1096,CC1CCCC(=O)CC=CC(CC(=O)O1)O,OCC\C=C/C[C@@H]1[C@H](CCC1=O)CC(O)=O,"NCGC00385243-01_C12H18O4_{(1R,2R)-2-[(2Z)-5-Hy...",227.1274,3.1773,1,OCC\C=C/C[C@@H]1[C@H](CCC1=O)CC(O)=O,MS2,Organic compounds,Lipids and lipid-like molecules,Fatty Acyls,Lineolic acids and derivatives,Jasmonic acids
349cf3d16890f62946648c49b3446a71,178,C=C1CC1CC(C(=O)O)N=C(C=CC(C(=O)O)N)O,missing,missing,291.0978,3.5890,1,C=C1CC1CC(C(=O)O)N=C(C=CC(C(=O)O)N)O,CSIFingerID,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",N-acyl-alpha amino acids
298983755e9288ee66227d8db396a5b7,349,CCCCCCCCCCCCNCC=O,missing,missing,228.2323,8.5250,1,CCCCCCCCCCCCNCC=O,CSIFingerID,Organic compounds,Organic nitrogen compounds,Organonitrogen compounds,Amines,Dialkylamines
91bc380746449edafc3b03c126d18701,448,CC(C1=CC=CC=C1)C(C)(C)COOC,missing,missing,209.1536,4.0212,1,CC(C1=CC=CC=C1)C(C)(C)COOC,CSIFingerID,Organic compounds,Benzenoids,Benzene and substituted derivatives,Phenylpropanes,Phenylpropanes
a5901fe3fea5c43752c0f96d0f9d2e60,499,CC1C(=O)C(C(C(O1)OC(=O)C=CC2=CC=C(C(=C2)OC)O)O)O,missing,missing,339.1072,3.4742,1,CC1C(=O)C(C(C(O1)OC(=O)C=CC2=CC=C(C(=C2)OC)O)O)O,CSIFingerID,Organic compounds,Phenylpropanoids and polyketides,Cinnamic acids and derivatives,Cinnamic acid esters,O-cinnamoyl glycosides


In [37]:
id_hash = classified_fdata.reset_index().set_index('#featureID')['id'].to_dict()
id_hash

{'1096': 'e459465fab00fe37f66bbd801e1abe23',
 '178': '349cf3d16890f62946648c49b3446a71',
 '349': '298983755e9288ee66227d8db396a5b7',
 '448': '91bc380746449edafc3b03c126d18701',
 '499': 'a5901fe3fea5c43752c0f96d0f9d2e60',
 '423': '949a198aa26adaa25d5c4aa2f888c966',
 '163': 'b503ea64bc37a69201885093ed93c809',
 '1149': '8b4d66803be55f907a59f42e8cd0162c',
 '693': '2498e4d3d62fb4b6d0b5f188bb6e0be7',
 '1074': '9a2e598944b8f3394bb13e358aa32ede',
 '245': '0bd5a212173a14d0701cd430e05556a1',
 '934': '057dac1c3ed84ce7876b9a3fb006235e',
 '585': '42d72655676bb854676249e42c260bb3',
 '1767': '70a64fe72d87e9e048dde83db0122307',
 '607': '41bd0d5f27755a29de19eef81aadb496',
 '459': 'f6611930822690b8888a1153ccc8ab6a',
 '980': 'f42c2fd1140009fc67251b118ecdf176',
 '1800': 'a1ed88127242b85c4dd8d5711284ac77',
 '1124': '7809389d2d14adfbfd4d9dcc170e29c0',
 '1127': '0e234ff852363736fad936ed79ecfdeb',
 '1932': 'cea544b939d6b1d3ed804f84482e473b',
 '651': 'b03a639477c80515d664c9ee2222d940',
 '1507': 'a3b7bd624ae39e

In [38]:
adjmat = pd.read_csv('data/comparison-to-cosine/GlobalFoodOmics/cosine_pairs.tsv', 
                     sep='\t', dtype=str)
adjmat['hash1'] = [id_hash[i] if i in id_hash.keys() else i for i in adjmat['CLUSTERID1']]
adjmat['hash2'] = [id_hash[i] if i in id_hash.keys() else i for i in adjmat['CLUSTERID2']]

# make pairs of all hashes
adjmat['pairs'] = list(zip(adjmat.hash1, adjmat.hash2))
adjmat.head(5)

,CLUSTERID1,CLUSTERID2,DeltaMZ,MinRatio,Cosine,AlignScore2,AlignScore3,hash1,hash2,pairs
0,1,2,86.925,0.000,0.0573,0.1021,1,013e50de34171b1810f7983301885f84,f684ed7f32e96e0f65d4e7bdcdfbb560,"(013e50de34171b1810f7983301885f84, f684ed7f32e..."
1,1,3,15.974,0.000,0.8594,0.4685,1,013e50de34171b1810f7983301885f84,62ac8ed056a2921bec7905ffbe10cf4f,"(013e50de34171b1810f7983301885f84, 62ac8ed056a..."
2,1,5,176.957,0.000,0.0032,0.0116,1,013e50de34171b1810f7983301885f84,59f72b5d4101010e32e4a27c83039951,"(013e50de34171b1810f7983301885f84, 59f72b5d410..."
3,1,6,-342.116,0.000,0.0144,0.0496,1,013e50de34171b1810f7983301885f84,6,"(013e50de34171b1810f7983301885f84, 6)"
4,1,7,-326.142,0.000,0.0189,0.0117,1,013e50de34171b1810f7983301885f84,7,"(013e50de34171b1810f7983301885f84, 7)"


In [39]:
# make pairs of all hashes
adjmat['pairs'] = list(zip(adjmat.hash1, adjmat.hash2))
adjmat.head(5)

,CLUSTERID1,CLUSTERID2,DeltaMZ,MinRatio,Cosine,AlignScore2,AlignScore3,hash1,hash2,pairs
0,1,2,86.925,0.000,0.0573,0.1021,1,013e50de34171b1810f7983301885f84,f684ed7f32e96e0f65d4e7bdcdfbb560,"(013e50de34171b1810f7983301885f84, f684ed7f32e..."
1,1,3,15.974,0.000,0.8594,0.4685,1,013e50de34171b1810f7983301885f84,62ac8ed056a2921bec7905ffbe10cf4f,"(013e50de34171b1810f7983301885f84, 62ac8ed056a..."
2,1,5,176.957,0.000,0.0032,0.0116,1,013e50de34171b1810f7983301885f84,59f72b5d4101010e32e4a27c83039951,"(013e50de34171b1810f7983301885f84, 59f72b5d410..."
3,1,6,-342.116,0.000,0.0144,0.0496,1,013e50de34171b1810f7983301885f84,6,"(013e50de34171b1810f7983301885f84, 6)"
4,1,7,-326.142,0.000,0.0189,0.0117,1,013e50de34171b1810f7983301885f84,7,"(013e50de34171b1810f7983301885f84, 7)"


In [40]:
cosines = adjmat.reset_index().set_index('pairs')['Cosine'].to_dict()
cosines

{('013e50de34171b1810f7983301885f84',
  'f684ed7f32e96e0f65d4e7bdcdfbb560'): '0.0573',
 ('013e50de34171b1810f7983301885f84',
  '62ac8ed056a2921bec7905ffbe10cf4f'): '0.8594',
 ('013e50de34171b1810f7983301885f84',
  '59f72b5d4101010e32e4a27c83039951'): '0.0032',
 ('013e50de34171b1810f7983301885f84', '6'): '0.0144',
 ('013e50de34171b1810f7983301885f84', '7'): '0.0189',
 ('013e50de34171b1810f7983301885f84',
  '5d73069589618fed892c3d05ed41c9ef'): '0.1070',
 ('013e50de34171b1810f7983301885f84',
  '5193b9039b47ed07bcd209626e1e0377'): '0.0945',
 ('013e50de34171b1810f7983301885f84',
  '1ff62723f996130ed6904b287cbfc783'): '0.0458',
 ('013e50de34171b1810f7983301885f84',
  '74011b5b5b73e98e6850901e4959989f'): '0.0179',
 ('013e50de34171b1810f7983301885f84',
  '797b76b31f69573d56c2491494f15c82'): '0.0552',
 ('013e50de34171b1810f7983301885f84',
  '4540159ba85b573432fdf4bb4e4a9da5'): '0.0256',
 ('013e50de34171b1810f7983301885f84',
  '5c7514cecd249254a0d94879ddaecc9a'): '0.0278',
 ('013e50de34171b1810f

In [41]:
pairs = itertools.combinations(classified_fdata.index, 2)
pairs = [(pair[0], pair[1]) for pair in pairs]

In [42]:
# pandas empty object
data = pd.DataFrame(index=list(classified_fdata.index), columns=list(classified_fdata.index))
data.head()

,e459465fab00fe37f66bbd801e1abe23,349cf3d16890f62946648c49b3446a71,298983755e9288ee66227d8db396a5b7,91bc380746449edafc3b03c126d18701,a5901fe3fea5c43752c0f96d0f9d2e60,949a198aa26adaa25d5c4aa2f888c966,b503ea64bc37a69201885093ed93c809,8b4d66803be55f907a59f42e8cd0162c,2498e4d3d62fb4b6d0b5f188bb6e0be7,9a2e598944b8f3394bb13e358aa32ede,...,db298a1c937ed7c124e87c25c5fb5111,71cba2d0fac7fe8b41581728508a615f,3f4db7d0f3c442badcb894089e031334,3574e0ff46f63b2e8279e92e9d4aaee0,db280f0db49f433c5064fdb83323f355,14ffaa209ac741f77b4f5cb3f351b8d9,c7e0f222ee25941e66949c79fea18fab,20feca7c0b39b6a023dacfa629219079,6c8e046cdcd713b4d42346e46a7ce92f,db74723e5d047bfacd9856d788abf124
e459465fab00fe37f66bbd801e1abe23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349cf3d16890f62946648c49b3446a71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298983755e9288ee66227d8db396a5b7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91bc380746449edafc3b03c126d18701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a5901fe3fea5c43752c0f96d0f9d2e60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# pandas fill-in pairwise cosine similarities
for pair in pairs:
    if pair in cosines.keys():
        if float(cosines[pair]) < 0.3:
            continue
        else:
            data.loc[pair[0], pair[1]] = float(cosines[pair])
            data.loc[pair[1], pair[0]] = float(cosines[pair])
            
# numpy similarity matrix; set diagnoals = 1
np_data = np.asarray(data)
np.fill_diagonal(np_data, 1)

# pandas similarity df; set NaNs (i.e. cosines < 0.3) to 0
data2 = pd.DataFrame(data = np_data, index=list(classified_fdata.index), columns=list(classified_fdata.index))
data2 = data2.fillna(0)
data2.head()

,e459465fab00fe37f66bbd801e1abe23,349cf3d16890f62946648c49b3446a71,298983755e9288ee66227d8db396a5b7,91bc380746449edafc3b03c126d18701,a5901fe3fea5c43752c0f96d0f9d2e60,949a198aa26adaa25d5c4aa2f888c966,b503ea64bc37a69201885093ed93c809,8b4d66803be55f907a59f42e8cd0162c,2498e4d3d62fb4b6d0b5f188bb6e0be7,9a2e598944b8f3394bb13e358aa32ede,...,db298a1c937ed7c124e87c25c5fb5111,71cba2d0fac7fe8b41581728508a615f,3f4db7d0f3c442badcb894089e031334,3574e0ff46f63b2e8279e92e9d4aaee0,db280f0db49f433c5064fdb83323f355,14ffaa209ac741f77b4f5cb3f351b8d9,c7e0f222ee25941e66949c79fea18fab,20feca7c0b39b6a023dacfa629219079,6c8e046cdcd713b4d42346e46a7ce92f,db74723e5d047bfacd9856d788abf124
e459465fab00fe37f66bbd801e1abe23,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349cf3d16890f62946648c49b3446a71,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298983755e9288ee66227d8db396a5b7,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91bc380746449edafc3b03c126d18701,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.3809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a5901fe3fea5c43752c0f96d0f9d2e60,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# tree construction using cosine dissimilarity
dm_cos = 1-np.asarray(data2) # cosine dissimilarity
distsq = squareform(dm_cos, checks=False)
linkage_matrix = linkage(distsq, method='average')
tree_cosine = TreeNode.from_linkage_matrix(linkage_matrix, data2.columns.tolist())
tree_cosine.write('data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.nwk')

! qiime tools import --input-path 'data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.nwk' \
 --output-path 'data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.qza' \
 --type Phylogeny[Rooted]

Imported data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.nwk as NewickDirectoryFormat to data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.qza


In [45]:
# unifrac using cosine-based tree
! qiime diversity beta-phylogenetic \
  --i-table data/comparison-to-cosine/GlobalFoodOmics/feature_table_hashed.qza \
  --i-phylogeny data/comparison-to-cosine/GlobalFoodOmics/tree_cosine.qza \
  --p-metric 'weighted_normalized_unifrac' \
  --o-distance-matrix data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_cosine.qza

Saved DistanceMatrix % Properties('phylogenetic') to: data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_cosine.qza


In [4]:
# bray-curtis (no tree)
! qiime diversity beta \
  --i-table data/comparison-to-cosine/GlobalFoodOmics/feature_table_hashed.qza \
  --p-metric 'braycurtis' \
  --o-distance-matrix data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_bray_curtis.qza

Saved DistanceMatrix to: data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_bray_curtis.qza


In [14]:
cosine_dm = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_cosine.qza').view(DistanceMatrix)
qem_dm = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/'
                       'Qemistree/output_folder/distance-matrix.qza').view(DistanceMatrix)
bc_dm = Artifact.load('data/comparison-to-cosine/GlobalFoodOmics/distance_matrix_bray_curtis.qza').view(DistanceMatrix)

# mapping file
mapping = load_mf('data/comparison-to-cosine/GlobalFoodOmics/'
                  'FBMN/qiime2_output/qiime2_metadata.tsv', 'sample_name')
mappingf = mapping.loc[list(cosine_dm.ids)]

In [15]:
# sample type group 1
col = 'sample_type_group1'
labels = mappingf[col].values

print('         ****COSINE-TREE****')
print(permanova(cosine_dm, labels))

print('         ****FINGERPRINT TREE****')
print(permanova(qem_dm, labels))

print('         ****NO TREE****')
print(permanova(bc_dm, labels))

         ****COSINE-TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              4.82414
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****FINGERPRINT TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              5.34768
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****NO TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              4.33612
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


In [17]:
# sample type group 2
col = 'sample_type_group2'
labels = mappingf[col].values

print('         ****COSINE-TREE****')
print(permanova(cosine_dm, labels))

print('         ****FINGERPRINT TREE****')
print(permanova(qem_dm, labels))

print('         ****NO TREE****')
print(permanova(bc_dm, labels))

         ****COSINE-TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  6
test statistic              4.75484
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****FINGERPRINT TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  6
test statistic              5.15483
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****NO TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  6
test statistic              4.22387
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


In [18]:
# sample type group 3
col = 'sample_type_group3'
labels = mappingf[col].values

print('         ****COSINE-TREE****')
print(permanova(cosine_dm, labels))

print('         ****FINGERPRINT TREE****')
print(permanova(qem_dm, labels))

print('         ****NO TREE****')
print(permanova(bc_dm, labels))

         ****COSINE-TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 11
test statistic              4.73052
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****FINGERPRINT TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 11
test statistic              4.85554
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****NO TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 11
test statistic              4.28973
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


In [19]:
# sample type group 4
col = 'sample_type_group4'
labels = mappingf[col].values

print('         ****COSINE-TREE****')
print(permanova(cosine_dm, labels))

print('         ****FINGERPRINT TREE****')
print(permanova(qem_dm, labels))

print('         ****NO TREE****')
print(permanova(bc_dm, labels))

         ****COSINE-TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 23
test statistic                5.038
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****FINGERPRINT TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 23
test statistic              5.35642
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
         ****NO TREE****
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 23
test statistic              4.30789
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


### II. Comparison to CSCS 

CSCS was computed using [q2-cscs plugin](https://github.com/madeleineernst/q2-cscs)

In [51]:
! qiime cscs cscs \
  --p-css-edges data/comparison-to-cosine/GlobalFoodOmics/gfop_edges_translated.txt \
  --i-features data/comparison-to-cosine/GlobalFoodOmics/gfop_merged-feature-table.qza \
  --p-normalization \
  --o-distance-matrix data/comparison-to-cosine/GlobalFoodOmics/gfop_cscs_distance_matrix.qza \
  --p-cpus 5 --p-chunk 1000

Saved DistanceMatrix % Properties('phylogenetic') to: data/comparison-to-cosine/GlobalFoodOmics/gfop_cscs_distance_matrix.qza


In [52]:
#Load distances
cscs_dist = qiime2.Artifact.load("data/comparison-to-cosine/GlobalFoodOmics/gfop_cscs_distance_matrix.qza")
cscs_dist_dm = cscs_dist.view(DistanceMatrix)

#Filter mapping
mappingf = mapping.loc[list(cscs_dist_dm.ids)]

# sample_type_group1_labels
sample_type_group1_labels = mappingf["sample_type_group1"].values
print(permanova(cscs_dist_dm, sample_type_group1_labels))

# sample_type_group2_labels
sample_type_group2_labels = mappingf["sample_type_group2"].values
print(permanova(cscs_dist_dm, sample_type_group2_labels))

# sample_type_group3_labels
sample_type_group3_labels = mappingf["sample_type_group3"].values
print(permanova(cscs_dist_dm, sample_type_group3_labels))

# sample_type_group4_labels
sample_type_group4_labels = mappingf["sample_type_group4"].values
print(permanova(cscs_dist_dm, sample_type_group4_labels))

method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  5
test statistic              4.63819
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                  6
test statistic              4.53509
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups                 11
test statistic              4.74771
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object
method name               PERMANOVA
test statistic name        pseudo-F
sample size                     126
number of groups   